In [5]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

from api_keys import g_key

In [6]:
# Store csv created in part one into a DataFrame
city_data_df = pd.read_csv("cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,longyearbyen,78.2186,15.6401,8.60,43,75,10.36,SJ,1612164567
1,1,madzhalis,42.1217,47.8333,50.18,49,0,5.19,RU,1612164675
2,2,cidreira,-30.1811,-50.2056,72.55,90,100,4.88,BR,1612164550
3,3,ushuaia,-54.8000,-68.3000,50.00,76,75,13.80,AR,1612164467
4,4,tasiilaq,65.6145,-37.6368,14.00,62,75,1.14,GL,1612164666


# Humidity Heatmap

In [7]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Heatmap of humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=5)

fig.add_layer(heat_layer)

fig 

Figure(layout=FigureLayout(height='420px'))

# Create new data frame 

In [8]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_city_df = city_data_df.loc[(city_data_df["Max Temp"] < 80) & (city_data_df["Max Temp"] > 70) \
                                    & (city_data_df["Wind Speed"] < 10) \
                                    & (city_data_df["Cloudiness"] == 0)].dropna()
narrowed_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
33,33,bandar-e lengeh,26.5579,54.8807,73.40,49,0,6.91,IR,1612164553
68,68,sao filipe,14.8961,-24.4956,72.45,67,0,6.33,CV,1612164551
76,76,conceicao do araguaia,-8.2578,-49.2647,72.30,93,0,0.87,BR,1612164685
108,108,dalmau,26.0667,81.0333,73.56,23,0,5.44,IN,1612164688
171,171,lano,10.1725,11.4715,78.04,18,0,5.32,NG,1612164696
224,224,tura,25.5198,90.2201,74.05,34,0,5.75,IN,1612164666
240,240,boa vista,2.8197,-60.6733,77.00,78,0,9.22,BR,1612164509


# Hotel map
##Store into variable named hotel_df.
Add a "Hotel Name" column to the DataFrame.
Set parameters to search for hotels with 5000 meters.
Hit the Google Places API for each city's coordinates.
Store the first Hotel result into the DataFrame.
Plot markers on top of the heatmap.

In [9]:
# Create DataFrame called hotel_df to store hotel names along with city, country and coordinates
hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
33,bandar-e lengeh,IR,26.5579,54.8807,
68,sao filipe,CV,14.8961,-24.4956,
76,conceicao do araguaia,BR,-8.2578,-49.2647,
108,dalmau,IN,26.0667,81.0333,
171,lano,NG,10.1725,11.4715,
224,tura,IN,25.5198,90.2201,
240,boa vista,BR,2.8197,-60.6733,


In [10]:
# Set parameters to search for a hotel
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Iterate through 
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotel_df

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


,City,Country,Lat,Lng,Hotel Name
33,bandar-e lengeh,IR,26.5579,54.8807,
68,sao filipe,CV,14.8961,-24.4956,
76,conceicao do araguaia,BR,-8.2578,-49.2647,
108,dalmau,IN,26.0667,81.0333,
171,lano,NG,10.1725,11.4715,
224,tura,IN,25.5198,90.2201,
240,boa vista,BR,2.8197,-60.6733,


In [11]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig 

Figure(layout=FigureLayout(height='420px'))